In [65]:
# Importing the required libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import sqlite3 as sql
import numpy as np
from datetime import datetime

In [66]:
"""Project Scenario
A multi-national firm has hired you as a data engineer. Your job is to access and process data as per requirements.

Your boss asked you to compile the list of the top 10 largest banks in the world ranked by market capitalization in 
billion USD. Further, you need to transform the data and store it in USD, GBP, EUR, and INR per the exchange rate 
information made available to you as a CSV file. You should save the processed information table locally in a CSV 
format and as a database table. Managers from different countries will query the database table to extract the list 
and note the market capitalization value in their own currency.

Directions
Write a function to extract the tabular information from the given URL under the heading By Market Capitalization, 
and save it to a data frame.
Write a function to transform the data frame by adding columns for Market Capitalization in GBP, EUR, and INR, 
rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.
Write a function to load the transformed data frame to an output CSV file.
Write a function to load the transformed data frame to an SQL database server as a table.
Write a function to run queries on the database table.
Run the following queries on the database table:
a. Extract the information for the London office, that is Name and MC_GBP_Billion
b. Extract the information for the Berlin office, that is Name and MC_EUR_Billion
c. Extract the information for New Delhi office, that is Name and MC_INR_Billion
Write a function to log the progress of the code.
While executing the data initialization commands and function calls, maintain appropriate log entries."""

'Project Scenario\nA multi-national firm has hired you as a data engineer. Your job is to access and process data as per requirements.\n\nYour boss asked you to compile the list of the top 10 largest banks in the world ranked by market capitalization in \nbillion USD. Further, you need to transform the data and store it in USD, GBP, EUR, and INR per the exchange rate \ninformation made available to you as a CSV file. You should save the processed information table locally in a CSV \nformat and as a database table. Managers from different countries will query the database table to extract the list \nand note the market capitalization value in their own currency.\n\nDirections\nWrite a function to extract the tabular information from the given URL under the heading By Market Capitalization, \nand save it to a data frame.\nWrite a function to transform the data frame by adding columns for Market Capitalization in GBP, EUR, and INR, \nrounded to 2 decimal places, based on the exchange rate

In [67]:
# Code for ETL operations on Country-GDP data
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    timestamp_format='%Y-%h-%d-%H:%M:%S' 
    now=datetime.now()
    timestamp=now.strftime(timestamp_format)
    with open ("log_file.txt","a") as f:
        f.write(f"{timestamp}: {message}\n")

def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    http=requests.get(url).text
    data=bs(http, "html.parser")
    table=data.find_all("tbody")[0]
    rows=table.find_all("tr")
    df=pd.DataFrame(columns=table_attribs)
    i=0
    for n in rows: 
        column=n.find_all("td")
        if i>0:
            temp_dict={"Name":column[1].find_all("a")[1].contents[0],"MC_USD_Billion":float(column[2].contents[0])}
            temp_df=pd.DataFrame(temp_dict, index=[0])
            df=pd.concat([df,temp_df],ignore_index=True)
        i+=1
    return df

def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    #Getting rates as variables
    Exchange_rate_CSV_df=pd.read_csv(csv_path)

    Eur_rate=float(Exchange_rate_CSV_df[Exchange_rate_CSV_df["Currency"]=="EUR"]["Rate"])
    Gbp_rate=float(Exchange_rate_CSV_df[Exchange_rate_CSV_df["Currency"]=="GBP"]["Rate"])
    Inr_rate=float(Exchange_rate_CSV_df[Exchange_rate_CSV_df["Currency"]=="INR"]["Rate"])

    #Transformin new columns values
    df["MC_GBP_Billion"]=round(df["MC_USD_Billion"]*Gbp_rate,2)
    df["MC_EUR_Billion"]=round(df["MC_USD_Billion"]*Eur_rate,2)
    df["MC_INR_Billion"]=round(df["MC_USD_Billion"]*Inr_rate,2)
    
    return df

def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path)

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    df.to_sql(table_name,sql_connection,if_exists='replace', index=False)

def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    return pd.read_sql(query_statement, sql_connection)

In [68]:
#DATI PRELIMINARI
url="https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
Exchange_rate_CSV="./exchange_rate.csv"
Table_Attributes_extraction=["Name", "MC_USD_Billion"]
Table_Attributes=["Name", "MC_USD_Billion", "MC_GBP_Billion", "MC_EUR_Billion", "MC_INR_Billion"]
Output_CSV_Path="./Largest_banks_data.csv"
Database_name="Banks.db"
Table_name="Largest_banks"
Log_file="code_log.txt"

log_progress("Preliminaries complete. Initiating ETL process") #Log
df=extract(url,Table_Attributes_extraction)

log_progress("Data extraction complete. Initiating Transformation process") #Log
df=transform(df, "./exchange_rate.csv")

log_progress("Data transformation complete. Initiating Loading process") #Log
load_to_csv(df,Output_CSV_Path)
log_progress("Data saved to CSV file") #Log

conn=sql.connect(Database_name)
log_progress("SQL Connection initiated") #Log
load_to_db(df,conn,Table_name)
log_progress("Data loaded to Database as a table, Executing queries") #Log

#Queries
run_query("SELECT * FROM Largest_banks", conn)
run_query("SELECT AVG(MC_GBP_Billion) FROM Largest_banks", conn)
run_query("SELECT Name from Largest_banks LIMIT 5", conn)

log_progress("Process Complete") #Log
conn.close()
log_progress("Server Connection closed") #Log


In [69]:
conn=sql.connect(Database_name)

In [70]:
run_query("SELECT * FROM Largest_banks", conn)

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,346.34,402.62,35910.71
1,Bank of America,231.52,185.22,215.31,19204.58
2,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
3,Agricultural Bank of China,160.68,128.54,149.43,13328.41
4,HDFC Bank,157.91,126.33,146.86,13098.63
5,Wells Fargo,155.87,124.70,144.96,12929.42
6,HSBC Holdings PLC,148.90,119.12,138.48,12351.26
7,Morgan Stanley,140.83,112.66,130.97,11681.85
8,China Construction Bank,139.82,111.86,130.03,11598.07
9,Bank of China,136.81,109.45,127.23,11348.39


In [71]:
run_query("SELECT AVG(MC_GBP_Billion) FROM Largest_banks", conn)

,AVG(MC_GBP_Billion)
0,151.987


In [72]:
run_query("SELECT Name from Largest_banks LIMIT 5", conn)

,Name
0,JPMorgan Chase
1,Bank of America
2,Industrial and Commercial Bank of China
3,Agricultural Bank of China
4,HDFC Bank


In [73]:
df

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,346.34,402.62,35910.71
1,Bank of America,231.52,185.22,215.31,19204.58
2,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
3,Agricultural Bank of China,160.68,128.54,149.43,13328.41
4,HDFC Bank,157.91,126.33,146.86,13098.63
5,Wells Fargo,155.87,124.70,144.96,12929.42
6,HSBC Holdings PLC,148.90,119.12,138.48,12351.26
7,Morgan Stanley,140.83,112.66,130.97,11681.85
8,China Construction Bank,139.82,111.86,130.03,11598.07
9,Bank of China,136.81,109.45,127.23,11348.39


In [74]:
df['MC_EUR_Billion'][4]

146.86

In [75]:
df.to_csv(Output_CSV_Path)